# 2-Class Logistic Regression

In [2]:
import numpy as np
from sklearn import preprocessing
import random
import math
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from decimal import Decimal
from sklearn import linear_model
# from sklearn import cross_validation

In [4]:
def load_data(filename,delim,dtyp):
    data = np.loadtxt(filename,delimiter=delim,dtype=dtyp)
    return data
    

In [5]:
def preprocess_2class(data,x_index,dtype):
    if x_index:
        X = data[:x_index,:-1]
    else:
        X = data[:,:-1]
    
    if dtype=='string':
        x = []
        for i in X:
            temp = []
            for feature in i:
                temp.append(eval(feature))
            x.append(temp)
        X = np.array(x)
    else:
        X = np.array(X)
        
    X = np.insert(X,0,1,axis=1)
    X = preprocessing.normalize(X)
    
    if x_index:
        y = data[:x_index,-1]
    else:
        y = data[:,-1]
    le = preprocessing.LabelEncoder()
    Y = le.fit_transform(y)
    return X,Y

In [15]:
def sigmoid_function(theta,X):
    '''
    input: theta: theta vector(1 x n)
           X:     data matrix (m x n)
   
    output: dictionary that maps the individual x vector index with it's sigmoid function value
    '''
    sigmoid_dict = dict()
    for ind,x in enumerate(X):
        z = Decimal(np.dot(theta,x.T))
        print 'z', z
        print 'expz', math.exp(-z)
        h = (1./(1+(math.exp(-z))))
#         print 'h',h
        sigmoid_dict[ind] = h
    return sigmoid_dict

In [7]:
def new_theta(theta,learning_rate,sigmoid,Y,X):
    update = 0
    for i in range(len(Y)):
        update += (sigmoid[i] - Y[i])*(X[i])
    return(theta-(learning_rate*update))

In [8]:
def likelihood(X,Y,theta):
    s = 0
    sig = sigmoid_function(theta,X)
    for ind,x in enumerate(X):
        h = sig[ind]
#         print 'sigmoid', h
        s += (Y[ind]*math.log(h)) + ((1-Y[ind])*(math.log(1-h)))
    return s

In [9]:
def predict_y(X,Y,x_predict,lrate,threshold,theta0,iterations):
    count = 0
    y_predict = []
    k = float('inf')
    old_theta = theta0
#     print 'theta0', theta0
    while k > threshold and count < iterations:
        
        sig = sigmoid_function(old_theta,X)
        nw_theta = new_theta(old_theta,lrate,sig,Y,X)
        likelihood0 = likelihood(X,Y,old_theta)
        likelihood1 = likelihood(X,Y,nw_theta)
        k = abs(likelihood1 - likelihood0)
        old_theta = nw_theta
        count += 1
    
    sigmoid = sigmoid_function(nw_theta,x_predict)
    for s in sigmoid:
        if s > 0.5:
            y_predict.append(1)
        if s < 0.5:
            y_predict.append(0)
    
    return y_predict

In [11]:
def cross_validation(filename,delim,dtype,x_index,k,learning_rate,iterations,error_threshold,verbose=True):
    data = load_data(filename,delim,dtype)
    X,Y = preprocess_2class(data,x_index,dtype)
    theta0 = np.array([np.random.uniform(0.001,0.5) for i in range(X.shape[1])])
    accuracy = []
    recall = []
    precision = []
    f_measure = []
    skl_accuracy = []
    fold = 0
    
    for train_ind, test_ind in KFold(X.shape[0],k,shuffle=True,random_state=10):
        X_train = X[train_ind]
        Y_train = Y[train_ind]
        X_test =  X[test_ind]
        Y_test =  Y[test_ind]
        Y_predict = predict_y(X_train,Y_train,X_test,learning_rate,error_threshold,theta0,iterations)
        
#         print 'Y_test', Y_test
#         print 'Y_pred', Y_predict
#         print ''
        
        model = linear_model.LogisticRegression()
        model.fit(X_train,Y_train)
        y_predict = model.predict(X_test)
        
        acc = accuracy_score(Y_test,Y_predict,normalize=True, sample_weight=None)
        c_matrix = confusion_matrix(Y_test, Y_predict)
        prec = precision_score(Y_test, Y_predict) 
        rec = recall_score(Y_test, Y_predict)  
        fm = f1_score(Y_test, Y_predict)
        
        accu = accuracy_score(Y_test,y_predict,normalize=True,sample_weight=None)
        skl_accuracy.append(accu)
    
        accuracy.append(acc)
        recall.append(rec)
        precision.append(prec)
        f_measure.append(fm)
        
        if verbose:
            print 'fold:', fold
            print 'accuracy:', acc
            print 'confusion_matrix'
            print c_matrix
            print 'precision', prec
            print 'recall' , rec
            print ' '
            fold += 1
    
    avg_acc = sum(accuracy)/len(accuracy)
    avg_pre = sum(precision)/len(precision)
    avg_rec = sum(recall)/len(recall)
    avg_fm = sum(f_measure)/len(f_measure)
    avg_skl_acc = sum(skl_accuracy)/len(skl_accuracy)
    
    print 'avg_accuracy: ' , avg_acc
    print 'sklearn_accuracy', avg_skl_acc
    print ''
    print 'avg_precision', avg_pre
    print 'avg_recall', avg_rec
    print 'avg_fmeasure', avg_fm

In [9]:
def temp():
#     data = load_data('bezdekIris.data.txt')
    data = load_data('Skin_NonSkin.txt')
#     print data
    
    X,Y = preprocess_2class(data,100)
    print X.shape
#     filename = 'bezdekIris.data.txt'
#     cross_validation(X,Y,10,0.001,1000,0.1,True)
#     cross_validation(X,Y,10,0.001,1000,0.1,True)
    
  

    

In [156]:
temp()

SystemError: error return without exception set

In [12]:
cross_validation('bezdekIris.data.txt',',','string',100,10,0.01,1000,0.1,True)

fold: 0
accuracy: 0.4
confusion_matrix
[[1 6]
 [0 3]]
precision 0.333333333333
recall 1.0
 
fold: 1
accuracy: 0.7
confusion_matrix
[[1 3]
 [0 6]]
precision 0.666666666667
recall 1.0
 
fold: 2
accuracy: 0.8
confusion_matrix
[[1 2]
 [0 7]]
precision 0.777777777778
recall 1.0
 
fold: 3
accuracy: 0.3
confusion_matrix
[[1 7]
 [0 2]]
precision 0.222222222222
recall 1.0
 
fold: 4
accuracy: 0.6
confusion_matrix
[[1 4]
 [0 5]]
precision 0.555555555556
recall 1.0
 
fold: 5
accuracy: 0.9
confusion_matrix
[[1 1]
 [0 8]]
precision 0.888888888889
recall 1.0
 
fold: 6
accuracy: 0.6
confusion_matrix
[[1 4]
 [0 5]]
precision 0.555555555556
recall 1.0
 
fold: 7
accuracy: 0.7
confusion_matrix
[[1 3]
 [0 6]]
precision 0.666666666667
recall 1.0
 
fold: 8
accuracy: 0.5
confusion_matrix
[[1 5]
 [0 4]]
precision 0.444444444444
recall 1.0
 
fold: 9
accuracy: 0.5
confusion_matrix
[[1 5]
 [0 4]]
precision 0.444444444444
recall 1.0
 
avg_accuracy:  0.6
sklearn_accuracy 1.0

avg_precision 0.555555555556
avg_recall

In [ ]:
cross_validation('Skin_NonSkin.txt',None,'float',None,10,0.01,1000,0.1,True)

z 0.2713480682303839319757798875798471271991729736328125
expz 0.762351100021
z 0.271186264983242164561261233757250010967254638671875
expz 0.762474460885
z 0.271186264983242164561261233757250010967254638671875
expz 0.762474460885
z 0.27110031801536826900900223336066119372844696044921875
expz 0.762539996069
z 0.271186264983242164561261233757250010967254638671875
expz 0.762474460885
z 0.271186264983242164561261233757250010967254638671875
expz 0.762474460885
z 0.27163535125670190684132876413059420883655548095703125
expz 0.762132120946
z 0.27163535125670190684132876413059420883655548095703125
expz 0.762132120946
z 0.271700361249886335190950603646342642605304718017578125
expz 0.762082576353
z 0.271700361249886335190950603646342642605304718017578125
expz 0.762082576353
z 0.271700361249886335190950603646342642605304718017578125
expz 0.762082576353
z 0.27176288029369122245526568804052658379077911376953125
expz 0.762034933168
z 0.27124533185417354363977437969879247248172760009765625
expz 0.76242

# (b) Non-linear combination of inputs

In [21]:
def non_linear_combination(filename,delim,dtype,x_index,degree,k,learning_rate,error_threshold,iterations):
    data = load_data(filename,delim,dtype)
    X,Y = preprocess_2class(data,x_index,dtype)
    poly = preprocessing.PolynomialFeatures(degree)
    Z = poly.fit_transform(X)
#     cross_validation(Z,Y,10,0.001,1000,0.1,True)

    theta0 = np.array([np.random.uniform(0.001,0.5) for i in range(Z.shape[1])])
    accuracy = []
    recall = []
    precision = []
    f_measure = []
    skl_accuracy = []
    fold = 0
    
    for train_ind, test_ind in KFold(Z.shape[0],k,shuffle=True,random_state=10):
        Z_train = Z[train_ind]
        Y_train = Y[train_ind]
        Z_test =  Z[test_ind]
        Y_test =  Y[test_ind]
        Y_predict = predict_y(Z_train,Y_train,Z_test,learning_rate,error_threshold,theta0,iterations)
        
#         print 'Y_test', Y_test
#         print 'Y_pred', Y_predict
#         print ''
        
        model = linear_model.LogisticRegression()
        model.fit(X_train,Y_train)
        y_predict = model.predict(X_test)
        
        acc = accuracy_score(Y_test,Y_predict,normalize=True, sample_weight=None)
        c_matrix = confusion_matrix(Y_test, Y_predict)
        prec = precision_score(Y_test, Y_predict) 
        rec = recall_score(Y_test, Y_predict)  
        fm = f1_score(Y_test, Y_predict)
        
        accu = accuracy_score(Y_test,y_predict,normalize=True,sample_weight=None)
        skl_accuracy.append(accu)
    
        accuracy.append(acc)
        recall.append(rec)
        precision.append(prec)
        f_measure.append(fm)
        
        if verbose:
            print 'fold:', fold
            print 'accuracy:', acc
            print 'confusion_matrix'
            print c_matrix
            print 'precision', prec
            print 'recall' , rec
            print ' '
            fold += 1
    
    avg_acc = sum(accuracy)/len(accuracy)
    avg_pre = sum(precision)/len(precision)
    avg_rec = sum(recall)/len(recall)
    avg_fm = sum(f_measure)/len(f_measure)
    avg_skl_acc = sum(skl_accuracy)/len(skl_accuracy)
    
    print 'avg_accuracy: ' , avg_acc
    print 'sklearn_accuracy', avg_skl_acc
    print ''
    print 'avg_precision', avg_pre
    print 'avg_recall', avg_rec
    print 'avg_fmeasure', avg_fm

In [ ]:
non_linear_combination('bezdekIris.data.txt',',','string',100,2,10,0.01,0.01,1000)

z 0.995038116597687594122589871403761208057403564453125
expz 0.369709352213
z 1.006764082992396946991675577010028064250946044921875
expz 0.365399470918
z 1.0055912615599342796457449367153458297252655029296875
expz 0.365828270652
z 0.99427264722787123130132158621563576161861419677734375
expz 0.36999246174
z 1.0132410059557652726169862944516353309154510498046875
expz 0.363040454524
z 1.018929895318648082280788003117777407169342041015625
expz 0.360981021043
z 0.9999005318725331381557452914421446621417999267578125
expz 0.367916035271
z 1.018820352480896307412194801145233213901519775390625
expz 0.361020566095
z 0.9919455793357527806364259959082119166851043701171875
expz 0.370854461895
z 0.988908869551659908125884612672962248325347900390625
expz 0.371982350937
z 1.0038415735481149848595805451623164117336273193359375
expz 0.366468916294
z 0.99381192384146876950268278960720635950565338134765625
expz 0.370162965194
z 0.99692627322287641344900066542322747409343719482421875
expz 0.369011941667
z 

# (C) K-class discriminant

In [113]:
def preprocess_kclass(data):
    X = data[:,:-1]
    Y = data[:,-1]
    
    x = []
    for i in X:
        temp = []
        for feature in i:
            temp.append(eval(feature))
        x.append(temp)
    
    X = np.array(x)
    X = np.insert(x,0,1,axis=1)
    X = preprocessing.normalize(X)

    le = preprocessing.LabelEncoder()
    Y = le.fit_transform(Y)
    
    return X,Y

In [114]:
def indicator(Y,class_val):
    l = list()
    for idx,y in enumerate(Y):
        if y == class_val:
            l.append(1)
        else:
            l.append(0)
    return l

In [115]:
'''
    input: theta (dictionary of (label,respective theta))
            label (respective class label)
            X
    output: softmax_dict 
                        dictionary :
                        (x, dictionary(label,softmax_val)) :
                                            (each softmax value mapped to label for x vector)
'''
def softmax_function(theta,labels,X):
    d = dict()
    
    for i,x in enumerate(X):
        label_to_n = list()
        label_to_val = list()
        
        denominator = 0
        for l in labels:
            t = theta[l]
            a = np.exp(np.dot(t,x.T))
            label_to_n.append((l,a))
        
        for (label,n) in label_to_n:
            denominator += n
        
        for (label,n) in label_to_n:
            label_to_val.append((label,1.*n/denominator))
        
        d[i] = label_to_val
    
    return d
        

In [118]:
'''
input: 
        theta (dictionary: label -> theta)
        softmax (dictionary: x -> (label:val))
        , indicess 
        X,Y,learning_rate
output: 
        new_theta value after one iteration for a given label
'''
def calculate_ktheta(theta,learning_rate,softmax,Y,X,labels):
    update = 0
    result = dict()
    for l in labels:
        indices = indicator(Y,l)
        
        for i in range(len(Y)):
            update += (softmax[i][l][1] - indices[i])*(X[i])
        result[l] = (theta[l]-(learning_rate*update))
    return result

In [119]:
'''
    input: labels , softmax_dict (dictionary (label,softmax))
    output: likelihood value
'''
def log_likelihood(labels,softmax_dict,indices,X):
    s = 0
    
    for ind,x in enumerate(X):
        for l in labels:
            s += 1.*(indices[ind])*(np.log(softmax_dict[ind][l][1]))
    return s

In [120]:
'''
input: X_train,Y_train,x_predict,
        theta0 = dictionary of (label: theta),
        iterations, learning_rate, threshold

output: list of Y prediction
'''
def kclass_predict_y(X,Y,x_predict,lrate,threshold,theta0,iterations):
    count = 0
    old_theta = theta0
    labels = np.unique(Y)
    k = float('inf')
    
    indices_dict = dict()
    nw_theta = dict()
    softmax_dict = dict()
    y_predict = list()
    
    for label in labels:
        indices_dict[label] = indicator(Y,label)

    
    while (k > threshold) and (count < iterations):
       
        for i,label in enumerate(labels):
            if k > threshold:
                softmax_dict0 = softmax_function(old_theta,labels,X)
                
                nw_theta = calculate_ktheta(old_theta,lrate,softmax_dict0,Y,X,labels)
                likelihood0 = log_likelihood(labels,softmax_dict0,indices_dict[label],X)
                
                softmax_dict1 = softmax_function(nw_theta,labels,X)
                likelihood1 = log_likelihood(labels,softmax_dict1,indices_dict[label],X)
                
                k = abs(likelihood1 - likelihood0)
                old_theta[label] = nw_theta[label]
        
        count += 1
   
    softmax_dict = softmax_function(nw_theta,labels,x_predict)
    for ind,listt in softmax_dict.items():
        y = max(listt, key=lambda x: x[1])[0]
        y_predict.append(y)
        
    return y_predict

In [121]:
'''
indicator function
a whole range of Y for all the X's for a given class..... 
calculate k theta

'''
def do_experiment(filename,k,learning_rate,iterations,error_threshold,verbose):
    data = load_data(filename)
    X,Y = preprocess_kclass(data)
    labels = np.unique(Y)
    theta0 = dict()
    
    for label in labels:
        theta0[label] = np.array([np.random.uniform(0.001,0.5) for i in range(X.shape[1])])
        
    accuracy = []
    recall = []
    precision = []
    f_measure = []
    skl_accuracy = []
    fold = 1
    
    for train_ind, test_ind in KFold(X.shape[0],k,shuffle=True,random_state=10):
        X_train = X[train_ind]
        Y_train = Y[train_ind]
        X_test =  X[test_ind]
        Y_test =  Y[test_ind]
       
        Y_predict = kclass_predict_y(X_train,Y_train,X_test,learning_rate,error_threshold,theta0,iterations)
        
        acc = accuracy_score(Y_test,Y_predict,normalize=True, sample_weight=None)
        c_matrix = confusion_matrix(Y_test, Y_predict)
        prec = precision_score(Y_test, Y_predict,average = 'micro') 
        rec = recall_score(Y_test, Y_predict,average = 'micro')  
        fm = f1_score(Y_test, Y_predict,average = 'micro')
        
        accu = accuracy_score(Y_test,Y_predict,normalize=True,sample_weight=None)
        skl_accuracy.append(accu)
    
        accuracy.append(acc)
        recall.append(rec)
        precision.append(prec)
        f_measure.append(fm)
        
        if verbose:
            print 'fold:', fold
            print 'accuracy:', acc
            print 'confusion_matrix'
            print c_matrix
            print 'precision', prec
            print 'recall' , rec
            print ' '
            fold += 1
    
    avg_acc = sum(accuracy)/len(accuracy)
    avg_pre = sum(precision)/len(precision)
    avg_rec = sum(recall)/len(recall)
    avg_fm = sum(f_measure)/len(f_measure)
    avg_skl_acc = sum(skl_accuracy)/len(skl_accuracy)
    
    print 'avg_accuracy: ' , avg_acc
    print 'sklearn_accuracy', avg_skl_acc
    print ''
    print 'avg_precision', avg_pre
    print 'avg_recall', avg_rec
    print 'avg_fmeasure', avg_fm

In [105]:
do_experiment('bezdekIris.data.txt',10,0.01,1000,0.01,True)

X (150, 5)
Y (150,)
Y_predict
[0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

fold: 1
accuracy: 0.466666666667
confusion_matrix
[[5 0 0]
 [0 0 8]
 [0 0 2]]
precision 0.466666666667
recall 0.466666666667
 
Y_predict
[0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

fold: 2
accuracy: 0.666666666667
confusion_matrix
[[5 0 0]
 [0 0 5]
 [0 0 5]]
precision 0.666666666667
recall 0.666666666667
 
Y_predict
[0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

fold: 3
accuracy: 0.733333333333
confusion_matrix
[[4 0 0]
 [0 0 4]
 [0 0 7]]
precision 0.733333333333
recall 0.733333333333
 
Y_predict
[0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

fold: 4
accuracy: 0.533333333333
confusion_matrix
[[4 0 0]
 [0 0 7]
 [0 0 4]]
precision 0.533333333333
recall 0.533333333333
 
Y_predict
[0, 0, 0, 0, 0, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2]

fold: 5
accuracy: 0.933333333333
confusion_matrix
[[5 0 0]
 [0 2 1]
 [0 0 7]]
precision 0.933333333333
recall 0.933333333333
 
Y_predict
[0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2]

fol

In [103]:
a = {1:2, 2:3, 4:5}
print a.keys()

[1, 2, 4]


In [122]:
do_experiment('bezdekIris.data.txt',10,0.01,1000,0.01,False)

avg_accuracy:  0.753333333333
sklearn_accuracy 0.753333333333

avg_precision 0.753333333333
avg_recall 0.753333333333
avg_fmeasure 0.753333333333


In [ ]:
from sklearn.datasets import fetch_mldata
import numpy as np
import os
from pylab import *

mnist = fetch_mldata('MNIST original')
mnist.data.shape
mnist.target.shape
np.unique(mnist.target)

X, y = mnist.data / 255., mnist.target
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]
X_train.shape
y_train.shape
size=len(y_train)

## extract "3" digits and show their average"
ind = [ k for k in range(size) if y_train[k]==3 ]
extracted_images=X_train[ind,:]

mean_image=extracted_images.mean(axis=0)
imshow(mean_image.reshape(28,28), cmap=cm.gray)
show()from sklearn.datasets import fetch_mldata
import numpy as np
import os
from pylab import *

mnist = fetch_mldata('MNIST original')
mnist.data.shape
mnist.target.shape
np.unique(mnist.target)

X, y = mnist.data / 255., mnist.target
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]
X_train.shape
y_train.shape
size=len(y_train)

## extract "3" digits and show their average"
ind = [ k for k in range(size) if y_train[k]==3 ]
extracted_images=X_train[ind,:]

mean_image=extracted_images.mean(axis=0)
imshow(mean_image.reshape(28,28), cmap=cm.gray)
show()